In [3]:
!uv sync
%load_ext autoreload 
%autoreload 2  
# imported functions are re-imported if they are changed

Resolved 151 packages in 0.41ms
Audited 148 packages in 0.10ms
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
from pyorbital.orbital import Orbital
import datetime as dt
from functools import lru_cache

@lru_cache(maxsize=2)
def get_satellite_orbital(satellite_name):
    
    possible_options = ["TERRA", "AQUA"]
    if satellite_name not in possible_options:
        raise ValueError(f"Satellite name should be one of {possible_options}")

    return Orbital(satellite_name)

In [211]:
def get_modis_overpass_time(satellite_name:str, date:str, lat:float, lon:float, altitude_km=0.3, min_horizon_angle=45):
    import pandas as pd 
    import numpy as np
    from timezonefinder import TimezoneFinder

    timezone = TimezoneFinder().timezone_at(lng=lon, lat=lat) 

    satellite = get_satellite_orbital(satellite_name)

    dtobj = dt.datetime.strptime(date, "%Y-%m-%d")

    passes = satellite.get_next_passes(dtobj, 24, lon, lat, altitude_km, horizon=min_horizon_angle)
    
    # get overhead time - compared against https://oceandata.sci.gsfc.nasa.gov/overpass_pred/
    overhead_time = pd.to_datetime([p[2] for p in passes])

    df = pd.DataFrame(passes, columns=["rise_time", "fall_time", "overpass_time_UTC"])

    df['overpass_time_local'] = df['overpass_time_UTC'].dt.tz_localize('UTC').dt.tz_convert(timezone)
    df['satellite'] = satellite_name
    df['request_date'] = pd.to_datetime(dtobj)
    df['lat'] = lat
    df['lon'] = lon
    df['altitude_km'] = altitude_km
    df['horizon_angle_min'] = min_horizon_angle
    df['night_pass'] = [~is_daytime(t, lat, lon) for t in overhead_time]
    df['observer_angle'] = [np.around(satellite.get_observer_look(t, lon, lat, altitude_km)[1], 2) for t in overhead_time]

    order = ['satellite', 'request_date', 'lat', 'lon', 'altitude_km', 'horizon_angle_min', 'overpass_time_UTC', 'overpass_time_local', 'observer_angle', 'night_pass']
    df = df[order]
    
    return df


def is_daytime(time, lat, lon):
    """Returns True if it's daytime at the given location, False otherwise."""

    from datetime import datetime, timezone
    from pyorbital.astronomy import sun_zenith_angle

    zenith_angle = sun_zenith_angle(time, float(lon), float(lat))
    
    return zenith_angle < 90  # Daytime if the sun is above the horizon

In [212]:
get_modis_overpass_time("TERRA", "2025-12-22", lat=-34, lon=20)

,satellite,request_date,lat,lon,altitude_km,horizon_angle_min,overpass_time_UTC,overpass_time_local,observer_angle,night_pass
0,TERRA,2025-12-22,-34,20,0.3,45,2025-12-22 20:13:04.766174,2025-12-22 22:13:04.766174+02:00,83.86,True
